# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

`Суть проекта` - Даны данные компании "Мегалайн" по клиентам, которые пользуются тарифами:`Смарт` или `Ультра`. Компания хочет построить систему, способную предложить клиенту наиболее подходящий ему тариф:`Смарт` или `Ультра`. <br>
`Конечная цель проекта` - Выбрать наиболее точную модель и довести её до доли правильных ответов более чем 0.75.

## Откройте и изучите файл

Начнем с импортирования всех нужных библиотек:

In [1]:
import pandas as pd
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data)
print(data.info())
data.describe()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Разбейте данные на выборки

Отделим 50% данных для тестовой выборки:

In [3]:
data_train, data_test = train_test_split(data, test_size = 0.5, random_state = 12345)

В данном случае у нас целевым является столбец 'is_ultra':

In [4]:
features = data_test.drop('is_ultra', axis = 1)
target = data_test['is_ultra']

Отделим 25% данных для валидационной выборки:

In [5]:
features_val, features_test, target_val, target_test = train_test_split(features, target, test_size=0.50, random_state=12345)

In [6]:
features_train = data_train.drop('is_ultra', axis = 1)
target_train = data_train['is_ultra']

In [7]:
print(features_train.shape, features_val.shape, features_test.shape, target_train.shape, target_val.shape, target_test.shape)

(1607, 4) (803, 4) (804, 4) (1607,) (803,) (804,)


## Исследуйте модели

Поскольку строются модели для задачи 'Классификации' и в условии требуется построить наиболее качественную модель, то MSE в данном проекте не потребуется.

Начнем с модели `Случайный лес в регрессии`:

In [8]:
best_model = None
best_result_forest = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 1):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train) 
        predictions_val = model_forest.predict(features_val) 
        result = accuracy_score(target_val, predictions_val)
        if result > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result
            best_est = est
            best_depth = depth

print("Качество наилучшей модели на валидационной выборке:", best_result_forest, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Качество наилучшей модели на валидационной выборке: 0.8019925280199253 Количество деревьев: 16 Максимальная глубина: 10


Для повышения аккуратности и точности шаг в est сделал равным 1. В целом качество получилось неплохим, а количество деревьев всего 16.

Следующая модель: `Дерево решений в регрессии`:

In [9]:
best_model = None
best_result_tree = 0
for depth in range(1, 11):
    model_decision = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_decision.fit(features_train, target_train) 
    predictions = model_decision.predict(features_val)
    result = accuracy_score(target_val, predictions) 
    if result > best_result_tree:
        best_model_decision = model_decision
        best_result_tree = result
print("Качество лучшей модели на валидационной выборке:", best_result_tree)

Качество лучшей модели на валидационной выборке: 0.788293897882939


В случае `Дерева решений` качество модели было так же неплохим, но всё же уступающим `Случайному лесу`

Следующая модель: `Логистическая регрессия`:

In [10]:
model_logistic = LogisticRegression(random_state=12345)
model_logistic.fit(features_train, target_train) 
predictions = model_logistic.predict(features_val) 
result_logistic = accuracy_score(target_val, predictions) 

print("Качество модели на валидационной выборке:", result_logistic)

Качество модели на валидационной выборке: 0.7471980074719801


В данном случае качество модели вышло далеко не самым лучшим, уступающим всем двум предыдущим моделям.

#### Вывод

Исходя из проведенного иследования, можно распределить 3 модели по своим местам в ТОП 3:<br>

1. Случайный лес в регрессии. Качество = 0.8019<br>
2. Дерево решений. Качество = 0.7882<br>
3. Логистическая регрессия. Качество = 0.7471
    
Поэтому самой наилучшей моделью, хоть и не самой быстрой, является модель `Случайный лес в регрессии`

## Проверьте модель на тестовой выборке

Начнем так же как и в прошлый раз с модели `Случайный лес в регрессии`:

In [11]:
predictions_model_forest = best_model_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_model_forest)
print("Качество модели на тестовой выборке:", result_forest)

Качество модели на тестовой выборке: 0.8159203980099502


На тестовой выборке так же показывается неплохой результат.

Далее идет `Дерево решений`:

In [12]:
predictions_model_decision = best_model_decision.predict(features_test)
result_tree = accuracy_score(target_test, predictions_model_decision)
print("Качество модели на тестовой выборке:", result_tree)

Качество модели на тестовой выборке: 0.7972636815920398


В данном случае `Дерево решений` показывает качество незначительно выше, чем на валидационной выборке и так же очень неплохой результат.

Следующим идёт `Логистическая регрессия`:

In [13]:
predictions_model_logistic = model_logistic.predict(features_test)
result_logic = accuracy_score(target_test, predictions_model_logistic)
print("Качество модели на тестовой выборке:", result_logic)

Качество модели на тестовой выборке: 0.7674129353233831


Как и в случае с `Деревом решений`, качество модели `Логистическая регрессия` возросло но по прежнему остается худшим из трех проверенных.

#### Вывод

Как и при проверки моделей на валидационной выборке, в случае с тестовой выборкой ТОП 3 аналогичен, где лидирует модель `Случайного леса в регрессии`

## (бонус) Проверьте модели на адекватность

Точно не уверен как делается это задание и что понимается под случайной моделью, но надеюсь я правильно накопал в интернете)

In [14]:
dummy = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
dummy.fit(features_train, target_train)
dummy_score = dummy.score(features_test, target_test)
print('Качество случайной модели:', dummy_score)
print('Качество модели Случайного леса:', result_forest)

Качество случайной модели: 0.7101990049751243
Качество модели Случайного леса: 0.8159203980099502


Как я понимаю, сравнивая на адекватность надо смотреть больше или меньше значение нашей модели в сравнении со случайной моделью.

В случае с моделью `Случайного леса в регрессии` разница имеется и при этом в положительную сторону, поэтому можно сделать вывод, что данная модель адекватна.

Далее проверю на адекватность модель `Дерева решений`:

In [15]:
dummy = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
dummy.fit(features_train, target_train)
dummy_score = dummy.score(features_test, target_test)
print('Качество случайной модели:', dummy_score)
print('Качество модели Случайного леса:', result_tree)

Качество случайной модели: 0.7101990049751243
Качество модели Случайного леса: 0.7972636815920398


Так же адекватна.

In [16]:
dummy = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
dummy.fit(features_train, target_train)
dummy_score = dummy.score(features_test, target_test)
print('Качество случайной модели:', dummy_score)
print('Качество модели Случайного леса:', result_logic)

Качество случайной модели: 0.7101990049751243
Качество модели Случайного леса: 0.7674129353233831


В данном случае модель `Логической регрессии` тоже адекватна.

#### Вывод

Все три модели прошли проверку на адекватность.

P.S, поскольку до конца не ясно что конкретно требовалось в данном задании и как правильно следовало проводить анализ, как идея возможно стоило сделать проверку только модели `Случайного леса в регрессии` как самой лучшей по качеству в сравнении с DummyClassifier у которого будут разные значения у параметра `strategy`, не только `most_frequent`, но и те, что ниже:

- prior
- stratified
- uniform
- constant


# Финальный вывод

В результате данного исследования были проверены 3 модели: 
- Случайный лес в регрессии
- Дерево решений в регрессии
- Логистическая регрессия

В ходе исследования было выявлено, что самой качественной моделью является модель `Случайного леса в регрессии` как в валидационной выборке, так и в тестовой, а самой худшей по качеству моделью является модель `Логистической регрессии` как в тестовой так и в валидационной выборке.

Так же модели были проверены на адекватность, в результате чего все модели являлись адекватными.


##### Ответ для компании: 
Использовать модель `Случайного леса в регрессии` с гиперпараметрами: max_depth = 10, n_estimators = 16, random_state = 12345

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
